In [1]:
import json, os
from typing import Optional, Dict, Any, List
import utils as util

from scienceworld import ScienceWorldEnv
from textworld import EnvInfos

# from alfworld.agents.environment import get_environment
# import alfworld.agents.modules.generic as generic

# Textworld

### Test Data Creation

In [ ]:
num_gens = 5

# Define our inputs
save_path = f"{os.getcwd()}/generated_data"
sampling_args = [
    util.SamplingArgs(name="cooking_singlemove", max_samples_per_env=20, rollout_length=1, total_samples=25_000, max_step_overlap=0),
    util.SamplingArgs(name="cooking_multimove", max_samples_per_env=8, rollout_length=5, total_samples=10_000, max_step_overlap=1),
]
cooking_wrapper = util.Textworld_Cooking_Wrapper_Env()

# Generate our data
data_sampler = util.Textworld_Sampling_Manager(
    wrapper = cooking_wrapper,
    sampling_args=sampling_args
)

for i in num_gens:
    data_sampler.generate_data(save_path=save_path, max_iters=None)

[iter 10] cooking_singlemove: 192/25000 | cooking_multimove: 78/10000 | active=2
[iter 20] cooking_singlemove: 379/25000 | cooking_multimove: 154/10000 | active=2
[iter 30] cooking_singlemove: 565/25000 | cooking_multimove: 230/10000 | active=2
[iter 40] cooking_singlemove: 742/25000 | cooking_multimove: 302/10000 | active=2
[iter 50] cooking_singlemove: 935/25000 | cooking_multimove: 381/10000 | active=2
[iter 60] cooking_singlemove: 1135/25000 | cooking_multimove: 461/10000 | active=2
[iter 70] cooking_singlemove: 1335/25000 | cooking_multimove: 541/10000 | active=2
[iter 80] cooking_singlemove: 1525/25000 | cooking_multimove: 618/10000 | active=2
[iter 90] cooking_singlemove: 1705/25000 | cooking_multimove: 693/10000 | active=2
[iter 100] cooking_singlemove: 1896/25000 | cooking_multimove: 770/10000 | active=2
[iter 110] cooking_singlemove: 2094/25000 | cooking_multimove: 850/10000 | active=2
[iter 120] cooking_singlemove: 2294/25000 | cooking_multimove: 930/10000 | active=2
[iter 1

### Wrapper

In [2]:
cooking_wrapper = util.Textworld_Cooking_Wrapper_Env()

# Generate new game and load env in
cooking_wrapper.generate_new_game(randomize_gen_args=True)
cooking_wrapper.load_env()

while not cooking_wrapper.game_state.done:
    cooking_wrapper.step_env()
    if cooking_wrapper.game_state.moves == 10:
        print(cooking_wrapper.get_env_state(full_state=True))
    # else:
        # print(cooking_wrapper.get_env_state(full_state=False))

Cooking World Generation:
Random Gen - True; Recipe Length - 5; Take - 3; Go - 12; Open_ - True; Cook - True; Cut - True; Drop - True
[('Full Observation', "Move 10 | Score = 1/15\nActionable Verbs: ['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'examine', 'go', 'insert', 'inventory', 'lock', 'look', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']\nObjective: You are hungry! Let's cook a delicious meal. Check the cookbook in the kitchen for the recipe. Once done, enjoy your meal!\nLocation: Kitchen\nInventory: You are carrying: a block of cheese, a red onion and a raw purple potato.\nCurrent Observation: You find yourself in a kitchen. An usual one. I guess you better just go and list everything you see here.\n\nOh wow! Is that what I think it is? It is! It's a fridge. The fridge contains a raw pork chop. A closed oven, which looks conventional, is in the corner. As if things weren't amazing enough already, you can even see a table. But there isn't a thing on it. You ma

### Playable Textworld

In [ ]:
test_playable_tw = True

if test_playable_tw:
    play_textworld = True

    # Setup wrapper and env
    cooking_wrapper = util.Textworld_Cooking_Wrapper_Env()
    cooking_gen_args = {
        "recipe": 5,
        "take": 3,
        "go": 1,
        "open_": True,
        "cook": True,
        "cut": True,
        "drop": True
    }
    cooking_wrapper.generate_new_game(
       randomize_gen_args=False,
       game_gen_args=cooking_gen_args
    )
    cooking_wrapper.load_env()
    
    # Print initial state
    util.print_env_state(cooking_wrapper.get_env_state(full_state=True))
    
    # Main play loop
    done = False
    while not done:
        if play_textworld:
            command = input("> ")
        else:
            command = None
        if command == "stop":
            break
        done = cooking_wrapper.step_env(command)
        util.print_env_state(cooking_wrapper.get_env_state(full_state=False))

Full Observation: Move 0 | Score = 0/10
Actionable Verbs: ['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'examine', 'go', 'insert', 'inventory', 'lock', 'look', 'open', 'prepare', 'put', 'slice', 'take', 'unlock']
Objective: You are hungry! Let's cook a delicious meal. Check the cookbook in the kitchen for the recipe. Once done, enjoy your meal!
Location: Kitchen
Inventory: You are carrying: a red apple and a raw red potato.
Current Observation: Well, here we are in the kitchen.

You can see a closed fridge close by. You see an oven. You see a table. But the thing is empty. You see a counter. You see a raw purple potato, a raw yellow potato, a red hot pepper, a cookbook and a knife on the counter. You see a stove. The stove is conventional. But oh no! there's nothing on this piece of trash. Hm. Oh well



User: Examine cookbook


Environment: You open the copy of "Cooking: A Modern Approach (3rd Ed.)" and start reading:

Recipe #1
---------
Gather all following ingredients a

# Science World

In [2]:
# Pretty-print helpers (optional)
def hr(title: Optional[str] = None):
    print("\n" + "-"*90)
    if title:
        print(title)
        print("-"*90)

def pkey(d: Dict[str, Any], key: str, label: Optional[str] = None):
    if key in d:
        print(f"{label or key}: {d[key]}")

# Environment bootstrap (works with or without a custom JAR)
def init_env(jar_path: Optional[str] = None, env_step_limit: int = 100) -> ScienceWorldEnv:
    """
    If jar_path is None or "", ScienceWorldEnv uses the built-in jar.
    """
    env = ScienceWorldEnv("", jar_path or "", envStepLimit=env_step_limit)
    return env

def list_tasks(env: ScienceWorldEnv) -> List[str]:
    tasks = env.get_task_names()
    for i, t in enumerate(tasks):
        print(f"{i:2d}: {t}")
    return tasks

def load_task(
    env: ScienceWorldEnv,
    task_name: str,
    var_num: int = 0,
    simplifications: Optional[str] = "easy",
    generate_gold_path: bool = True
):
    """
    simplifications: e.g., "easy" or a comma-joined string like "teleportAction,openDoors,selfWateringFlowerPots,noElectricalAction"
    """
    env.load(task_name, var_num, simplifications or "", generateGoldPath=generate_gold_path)
    obs, info = env.reset()
    return obs, info

In [3]:
# Use default envs defined in science world
env = init_env(jar_path=None, env_step_limit=100)
tasks = list_tasks(env)

 0: boil
 1: change-the-state-of-matter-of
 2: chemistry-mix
 3: chemistry-mix-paint-secondary-color
 4: chemistry-mix-paint-tertiary-color
 5: find-animal
 6: find-living-thing
 7: find-non-living-thing
 8: find-plant
 9: freeze
10: grow-fruit
11: grow-plant
12: identify-life-stages-1
13: identify-life-stages-2
14: inclined-plane-determine-angle
15: inclined-plane-friction-named-surfaces
16: inclined-plane-friction-unnamed-surfaces
17: lifespan-longest-lived
18: lifespan-longest-lived-then-shortest-lived
19: lifespan-shortest-lived
20: measure-melting-point-known-substance
21: measure-melting-point-unknown-substance
22: melt
23: mendelian-genetics-known-plant
24: mendelian-genetics-unknown-plant
25: power-component
26: power-component-renewable-vs-nonrenewable-energy
27: test-conductivity
28: test-conductivity-of-unknown-substances
29: use-thermometer


In [4]:
# # Choose task to load / play around with
# task_idx = 6
# task_name = tasks[task_idx]
# print(f"Loading task #{task_idx}: {task_name}.")

# # Load task and print initial observation
# obs, info = load_task(env, task_name=task_name, var_num=1, simplifications="easy", generate_gold_path=True)
# print(f"Task: {env.get_task_description()}")
# print(f"Goal Progress: {env.get_goal_progress()}")
# gold_actions = env.get_gold_action_sequence()
# print(gold_actions)

# for idx, a in enumerate(gold_actions):
#     print(f"Step {idx+1}:\n{obs}")
#     print(f"Action: {a}")
#     obs, reward, isCompleted, infos = env.step(a)
#     print(f"Reward: {reward:.2f} | Score: {infos['score']:.2f} |  Completed? {isCompleted}\n\n")

In [5]:
play_scienceworld = False

if play_scienceworld:
    # Load task    
    task_idx = 8
    task_name = tasks[task_idx]
    print(f"Loading task #{task_idx}: {task_name}.")
    obs, info = load_task(env, task_name=task_name, var_num=1, simplifications="easy", generate_gold_path=True)
    print(f"Task: {env.get_task_description()}")
    score, moves, done = 0, 0, False
    
    # Render loop
    reward = 0
    while not done:
        command = input("> ")
        obs, score, done, infos = env.step(command)
        reward += score
        print(f"Observation:\n{obs}\n")
        print(f"Score: {score}; Total Reward: {reward}; Done: {done}\n####################\n\n")
        moves += 1

    print(f"Moves: {moves}; Reward: {reward}\n")
    print(f"Gold Path:\n{env.get_gold_action_sequence()}")
    print(f"Goal Progress:\n{env.get_goal_progress()}")
    env.reset()

# Text World

Global seed: 3351
Game generated: /home/lucas/code/seq-decision/sequential-decision-processors/data/textworld/raw/tw_games/custom_cooking.z8
{'objective': "You are hungry! Let's cook a delicious meal. Check the cookbook in the kitchen for the recipe. Once done, enjoy your meal!", 'verbs': ['chop', 'close', 'cook', 'dice', 'drink', 'drop', 'eat', 'examine', 'go', 'insert', 'inventory', 'lock', 'look', 'open', 'prepare', 'put', 'slice', 'take', 'unlock'], 'location': None, 'inventory': 'You are carrying: a red apple, a banana and a raw red potato.', 'description': "-= Kitchen =-\nThis is going to sound unbelievable, but you've just entered a kitchen. The room seems oddly familiar, as though it were only superficially different from the other rooms in the building.\n\nYou scan the room for a fridge, and you find a fridge. Suddenly, you bump your head on the ceiling, but it's not such a bad bump that it's going to prevent you from looking at objects and even things. You hear a noise behind